In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

"Why don't ice creams ever get into arguments?\n\nBecause they always stay cool!"

In [2]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [3]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [4]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

In [5]:
message = model.invoke(prompt_value)
message

AIMessage(content="Why don't ice creams ever get invited to parties?\n\nBecause they always melt under pressure!", response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 15, 'total_tokens': 33}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-35e31af6-b252-40c2-ae01-e0f958f03195-0')

In [6]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nRobot: Why did the ice cream go to the doctor? Because it was feeling a little melon-choly!'

In [7]:
output_parser.invoke(message)

"Why don't ice creams ever get invited to parties?\n\nBecause they always melt under pressure!"

### RAG 搜索示例

In [9]:
# 需要安装：
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

c:\Users\guojj\AppData\Local\pypoetry\Cache\virtualenvs\ai-langchain-tutorial-uYz6lp8f-py3.12\Lib\site-packages\docarray\helper.py:224: SyntaxWarning: invalid escape sequence '\*'
  """
c:\Users\guojj\AppData\Local\pypoetry\Cache\virtualenvs\ai-langchain-tutorial-uYz6lp8f-py3.12\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


'Harrison worked at Kensho.'

In [12]:
retriever.invoke("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [14]:
setup_and_retrieval.invoke("where did harrison work?")

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}